<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Agent/blob/main/autogen/L1_Multi_Agent_Conversation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 1: Multi-Agent Conversation and Stand-up Comedy

Welcome to Lesson 1.

To access the `requirements.txt` file and the`utils` modules, please go to the `File` menu and select`Open...`.

I hope you enjoy this course!

In [ ]:
!pip install -r requirements.txt

In [2]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=FILEID'  -O ./.env

--2024-06-10 20:04:17--  https://docs.google.com/uc?export=download&id=11_nDjppPKD6TOB3Zb1RpiAMafKmbc_c5
Resolving docs.google.com (docs.google.com)... 74.125.196.139, 74.125.196.100, 74.125.196.113, ...
Connecting to docs.google.com (docs.google.com)|74.125.196.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=11_nDjppPKD6TOB3Zb1RpiAMafKmbc_c5&export=download [following]
--2024-06-10 20:04:17--  https://drive.usercontent.google.com/download?id=11_nDjppPKD6TOB3Zb1RpiAMafKmbc_c5&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.196.132, 2607:f8b0:400c:c36::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.196.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 175 [application/octet-stream]
Saving to: ‘./.env’

./.env              100%[===================>]     175  --.-KB/s    in 0s      

2024-06

In [3]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
# Access the API key using the variable name defined in the .env file
api_key = os.getenv("OPENAI_API_KEY")

## Setup

In [4]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": "gpt-3.5-turbo"}

## Define an AutoGen agent

In [5]:
from autogen import ConversableAgent

agent = ConversableAgent(
    name="chatbot",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [6]:
reply = agent.generate_reply(
    messages=[{"content": "Tell me a joke.", "role": "user"}]
)
print(reply)

Why don't scientists trust atoms?

Because they make up everything!


In [7]:
reply = agent.generate_reply(
    messages=[{"content": "Repeat the joke.", "role": "user"}]
)
print(reply)

Certainly! Which joke would you like me to repeat for you?


## Conversation

Setting up a conversation between two agents, Cathy and Joe, where the memory of their interactions is retained.

In [8]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "Start the next joke from the punchline of the previous joke.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

**Note**: You might get a slightly different response (set of jokes) than what is shown in the video

In [9]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=2,
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe, great to meet you! So, I was at the gym the other day, trying to get into shape. I started doing crunches, and then I realized... that's why they're called crunches, because you can hear all the junk food you've eaten over the years yelling, "No, stop, not the abs!"

--------------------------------------------------------------------------------
joe (to cathy):

Haha, that's a good one! It's like your abs are having a little protest march in there.

--------------------------------------------------------------------------------
cathy (to joe):

Thanks, Joe! Exactly! My abs were definitely not happy with me that day. But hey, I like to think of it as a workout for my sense of humor too - the more I laugh at myself, the easier those sit-ups become!

---------------------------------------------------------------------

## Print some results

You can print out:

1. Chat history
2. Cost
3. Summary of the conversation

In [ ]:
import pprint

pprint.pprint(chat_result.chat_history)

[{'content': "I'm Joe. Cathy, let's keep the jokes rolling.",
  'role': 'assistant'},
 {'content': "Hey Joe! Buckle up, we're about to take this comedy show to the "
             'next level!',
  'role': 'user'},
 {'content': "Sounds good! Let's fasten our seat belts and enjoy the ride!",
  'role': 'assistant'},
 {'content': "That's the spirit, Joe! Just make sure those seat belts can "
             'handle all the laughter coming your way.',
  'role': 'user'}]


In [ ]:
pprint.pprint(chat_result.cost)

{'usage_excluding_cached_inference': {'total_cost': 0},
 'usage_including_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 57,
                                                             'cost': 0.00017950000000000003,
                                                             'prompt_tokens': 188,
                                                             'total_tokens': 245},
                                      'total_cost': 0.00017950000000000003}}


In [ ]:
pprint.pprint(chat_result.summary)

("That's the spirit, Joe! Just make sure those seat belts can handle all the "
 'laughter coming your way.')


## Get a better summary of the conversation

In [ ]:
chat_result = joe.initiate_chat(
    cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=2,
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Buckle up, we're about to take this comedy show to the next level!

--------------------------------------------------------------------------------
joe (to cathy):

Sounds good! Let's fasten our seat belts and enjoy the ride!

--------------------------------------------------------------------------------
cathy (to joe):

That's the spirit, Joe! Just make sure those seat belts can handle all the laughter coming your way.

--------------------------------------------------------------------------------


In [ ]:
pprint.pprint(chat_result.summary)

('Joe and Cathy are ready to keep the jokes rolling and enjoy a fun-filled '
 'comedy show together.')


## Chat Termination

Chat can be terminated using a termination conditions.

In [ ]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"] or "Goodbye" in msg["content"],
)

In [ ]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling."
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe, nice to meet you! I hope you're ready for some laughs. Why did the scarecrow win an award? Because he was outstanding in his field!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, that's a classic one, Cathy! Here's one for you: What do you call a can opener that doesn't work? A can't opener!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, I love it, Joe! Here's another one for you: Why did the math book look sad? Because it had too many problems.

--------------------------------------------------------------------------------
joe (to cathy):

Haha, that's a good one, Cathy! Here's one last joke before I gotta go: Why don't scientists trust atoms? Because they make up everything!

------------------------

In [ ]:
cathy.send(message="What's last joke we talked about?", recipient=joe)

cathy (to joe):

What's last joke we talked about?

--------------------------------------------------------------------------------
joe (to cathy):

The last joke I shared with you was: Why don't scientists trust atoms? Because they make up everything!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, that one's a classic! Thanks for the laughs, Joe. It was great joking around with you! If you ever need a good laugh, I'm here. And remember, laughter is the best medicine!

--------------------------------------------------------------------------------
joe (to cathy):

Thanks, Cathy! It was a blast exchanging jokes with you. Laughter truly is the best medicine. Take care and keep spreading those good vibes with your jokes! I gotta go.

--------------------------------------------------------------------------------
